In [1]:
import os
import pickle
from ase.db import connect
from ase.visualize import view

from GAMERNet.rnet.networks.reaction_network import ReactionNetwork
from GAMERNet.rnet.networks.surface import Surface

with open('../scripts/C1_ag100/rxn_net.pkl', 'rb') as pickle_file:
    content = pickle.load(pickle_file)  # dict of elementary reactions

rxn_net = ReactionNetwork().from_dict(content)
print(rxn_net)
graph = rxn_net.graph

ReactionNetwork(43 intermediates, 14 closed-shell molecules, 103 reactions)
Surface: Ag48(100)
Network Carbon cutoff: C1



In [5]:
rxn_net.t_states

[CH4+Ag48<->H+CH3,
 Ag48+CH3<->H+CH2,
 Ag48+CH2<->H+CH,
 CH+Ag48<->C+H,
 Ag48+CH4O2<->CH3O2+H,
 Ag48+CH4O2<->CH3O2+H,
 CH3O2+Ag48<->CH2O2+H,
 CH3O2+Ag48<->H+CH2O2,
 CH3O2+Ag48<->CH2O2+H,
 CH3O2+Ag48<->CH2O2+H,
 CH2O2+Ag48<->H+CHO2,
 CH2O2+Ag48<->CHO2+H,
 Ag48+CH2O2<->CHO2+H,
 CH2O2+Ag48<->H+CHO2,
 Ag48+CHO2<->CO2+H,
 CHO2+Ag48<->CO2+H,
 CH4O+Ag48<->H+CH3O,
 CH4O+Ag48<->CH3O+H,
 Ag48+CH3O<->H+CH2O,
 Ag48+CH3O<->H+CH2O,
 CH3O+Ag48<->H+CH2O,
 Ag48+CH2O<->H+CHO,
 Ag48+CH2O<->H+CHO,
 Ag48+CH2O<->H+CHO,
 Ag48+CHO<->CO+H,
 Ag48+CHO<->CO+H,
 CH4O3+Ag48<->CH3O3+H,
 CH4O3+Ag48<->H+CH3O3,
 CH3O3+Ag48<->CH2O3+H,
 CH3O3+Ag48<->CH2O3+H,
 Ag48+CH3O3<->CH2O3+H,
 CH2O3+Ag48<->CHO3+H,
 CH2O3+Ag48<->CHO3+H,
 CH2O3+Ag48<->CHO3+H,
 CHO3+Ag48<->CO3+H,
 CHO3+Ag48<->CO3+H,
 CH4O4+Ag48<->CH3O4+H,
 CH3O4+Ag48<->CH2O4+H,
 CH2O4+Ag48<->H+CHO4,
 Ag48+CHO4<->H+CO4,
 H2O+Ag48<->HO+H,
 HO+Ag48<->H+O,
 Ag48+H2O2<->HO2+H,
 HO2+Ag48<->O2+H,
 H2+Ag48<->H,
 Ag48+CH4O2<->HO+CH3O,
 CH3O2+Ag48<->CH3O+O,
 CH3O2+Ag48<->HO+CH2O

In [2]:
print(len(rxn_net.intermediates))

58


In [2]:
closed_shell_atoms = []
for inter in rxn_net.intermediates.values():
    if inter.closed_shell == True:
        closed_shell_atoms.append(inter.molecule)
print(len(closed_shell_atoms))

44


In [ ]:
closed_shell_atoms = []
for inter in rxn_net.intermediates.values():
    if inter.closed_shell == True:
        closed_shell_atoms.append(inter.molecule)
print(len(closed_shell_atoms))

In [4]:
closed_shell_atoms[0].get_chemical_symbols().count("H")

6

In [10]:
view(closed_shell_atoms)

<Popen: returncode: None args: ['/home/smorandi/miniconda3/envs/gnn/bin/pyth...>

In [3]:
y = rxn_net.gen_graph()
y

In [2]:
rxn_net.write_dotgraph(".", 'testCOLI.png')

KeyError: 'category'

Look for intermediates with specified composition

In [7]:
rxn_net.search_inter_by_elements({'C':1, 'H':2, 'O':2})

(122101(CH2O2), 122102(CH2O2), 122103(CH2O2))

Look for all elementary steps involving a specific intermediate

In [16]:
rxn_net.search_ts(["222101"])

(Ag48+C2H3O2<->C2H2O2+H,
 C2H3O2+Ag48<->C2H2O2+H,
 C2H3O2+Ag48<->C2H2O2+H,
 C2H2O2+Ag48<->H+C2HO2,
 C2H2O2+Ag48<->C2HO2+H,
 C2H3O3+Ag48<->HO+C2H2O2,
 C2H3O3+Ag48<->HO+C2H2O2,
 C2H2O3+Ag48<->O+C2H2O2,
 C2H2O3+Ag48<->O+C2H2O2,
 C2H2O2+Ag48<->O+C2H2O,
 C2H2O2+Ag48<->CH2O+CO,
 C2H2O2+Ag48<->C2HO+HO)

In [2]:
types = []
for reaction in rxn_net.t_states:
    types.append(reaction.r_type)
print(set(types))

{'C-O', 'H-H', 'O-O', 'C-OH', 'O-H', 'C-H', 'desorption'}


In [4]:
types = []
for inter in rxn_net.intermediates.values():
    types.append(inter.phase)
print(set(types))

{'surface', 'cat', 'gas'}


# closed shell

In [29]:
def is_closed_shell_santi(self):
        """
        Check if a molecule CxHyOz is closed-shell or not.
        """
        graph = self.graph
        # print(graph.nodes()) list of node indexes, element symbol stored as "elem"
        molecule = self.molecule
        valence_electrons = {'C': 4, 'H': 1, 'O': 2}
        graph = graph.to_undirected()
        mol_composition = molecule.get_chemical_symbols()
        mol = {'C': mol_composition.count('C'), 'H': mol_composition.count('H'), 'O': mol_composition.count('O')} # CxHyOz

        if mol['C'] != 0 and mol['H'] == 0 and mol['O'] == 0: # Cx
                return False
        elif mol['C'] == 0 and mol['H'] != 0 and mol['O'] == 0: # Hy
                return True if mol['H'] == 2 else False
        elif mol['C'] == 0 and mol['H'] == 0 and mol['O'] != 0: # Oz
                return True if mol['O'] == 2 else False
        elif mol['C'] != 0 and mol['H'] == 0 and mol['O'] != 0: # CxOz
                return True if mol['C'] == 1 and mol['O'] in (1,2) else False
        elif mol['C'] != 0 and mol['H'] != 0: # CxHyOz (z can be zero)
            node_val = lambda graph: {node: (graph.degree(node), 
                                        valence_electrons.get(graph.nodes[node]["elem"], 0)) for node in graph.nodes()}
            num_unsaturated_nodes = lambda dict: len([node for node in dict.keys() if dict[node][0] < dict[node][1]])
            node_valence_dict = node_val(graph)
            if num_unsaturated_nodes(node_valence_dict): # all atoms are saturated
                return True
            elif num_unsaturated_nodes(node_valence_dict) == 1: # only one unsaturated atom
                return False
            else:
                saturation_condition = lambda dict: all(dict[node][0] == dict[node][1] for node in dict.keys())
                while saturation_condition(node_valence_dict) == False:
                    unsat_nodes = [node for node in node_valence_dict.keys() if node_valence_dict[node][0] < node_valence_dict[node][1]]
                    O_unsat_nodes = [node for node in unsat_nodes if graph.nodes[node]["elem"] == 'O']  # all oxygens unsaturated
                    if len(O_unsat_nodes) != 0: # only one unsaturated oxygen
                        for oxygen in O_unsat_nodes:
                            node_valence_dict[oxygen][0] += 1
                            # increase the valence of the oxygen neighbour by 1
                            for neighbour in graph.neighbors(oxygen): # only one neighbour
                                if node_valence_dict[neighbour][0] < node_valence_dict[neighbour][1]:
                                    node_valence_dict[neighbour][1] += 1
                                else:
                                    return False # O neighbour is saturated already
                    else: # CxHy
                         # select node with the highest degree
                        max_degree = max([node_valence_dict[node][0] for node in unsat_nodes])
                        max_degree_node = [node for node in unsat_nodes if node_valence_dict[node][0] == max_degree][0]
                        max_degree_node_unsat_neighbours = [neighbour for neighbour in graph.neighbors(max_degree_node) if neighbour in unsat_nodes]
                        if len(max_degree_node_unsat_neighbours) == 0: # all neighbours are saturated
                            return False
                        node_valence_dict[max_degree_node][0] += 1
                        node_valence_dict[max_degree_node_unsat_neighbours][0] += 1
                         
                            

                
        
        # # Getting the unsaturated nodes (if there are not unsaturated nodes, the molecule is closed-shell)
        # unsat_nodes = [node for node in graph.nodes() if graph.degree(node) < valence_electrons.get(graph.nodes[node]["elem"], 0)]

        # # If the graph only has Carbon as an element and not H or O, then it is open-shell
        # if not 'H' and 'O' in molecule.get_chemical_formula():
        #     print(f'System {molecule.get_chemical_formula()} is open-shell: only C atoms')
        #     return False 
        
        # # Specific case for O2
        # if not 'C' and 'H' in molecule.get_chemical_formula() and len(unsat_nodes) == 2:
        #     print(f'System {molecule.get_chemical_formula()} is closed-shell: Oxygen')
        #     return True 
        
        # # CO and CO2
        # if not 'H' in molecule.get_chemical_formula() and len(molecule.get_chemical_symbols()['C']) == 1 and len(molecule.get_chemical_symbols()['O']) in (1,2):
        #     print(f'System {molecule.get_chemical_formula()} is closed-shell: CO or CO2')
        #     return True
        
        # if unsat_nodes:
        #     # If the molecule has only one unsaturated node, then it is open-shell
        #     if len(unsat_nodes) == 1:
        #         print(f'System {molecule.get_chemical_formula()} is open-shell')
        #         return False 
        #     else:
        #         # Checking if there is one unsaturated node that does not have as neighbour another unsaturated node
        #         for node in unsat_nodes:
        #             # If the molecule has only one unsaturated node, then it is open-shell
        #             if not [n for n in graph.neighbors(node) if n in unsat_nodes]:
        #                 print(f'System {molecule.get_chemical_formula()} is open-shell: one node is unsaturated but does not have as neighbour another unsaturated node')
        #                 return False 
        #             else:
        #                 # Case for molecules where an unsaturated node is oxygen
        #                 if graph.nodes[node]["elem"] == 'O':
        #                     # Adding one bond order (valence electrons) to the oxygen node by adding it to the unsat_nodes list

In [30]:
is_closed_shell_santi(rxn_net.intermediates['121101'])

True